# Hate Speech and Offensive Language Detection using BERT

The aim of this project is to detect whether a text is considered a hate speech, an offensive language, or neither offensive nor non-offensive using BERT model

**Warning:** It's important to note that dataset contains text that be considered offensive, sexist, discriminative, and racist. Dataset usage is purely for the research purpose.

## Install Requirements

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.1 MB/s eta 0:00:00
     

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, DistilBertTokenizer,BertForSequenceClassification, DistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn as nn

## Dataset Preparation and Preprocessing

Dataset is from https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset using Twitter data. Text is classified as

0: hate-speech
1: offensive-language
2: neither

In [ ]:
df = pd.read_csv('labeled_data.csv')

#Merge tweet that has more than 1 row
df['tweet'] = df['tweet'].replace('\n', ' ', regex=True)
# Data cleaning: Remove rows with missing values in any column
df = df.dropna(subset=['count','hate_speech','offensive_language','neither','tweet', 'class'])

df.to_csv('preprocessed_data.csv', index=False)

# Display the first few rows of the dataframe
print(df.head())
print(df.index)


   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  
RangeIndex(start=0, stop=24783, step=1)


Columns in this dataset include:
1. count: number of users who quoted the tweet
2. hate_speech: number of users who judged the tweet to be hate speech
3. offensive_language: number if uses who judged the tweet to be offensive language
4. neither: number of users who judged the tweet to be neither offensive nor non-offensive
5. class: tweet category, either it is considered hate-speech(0), offensive language(1), or neither offensive nor non-offensive(2)
6. tweet: the users tweet

### Train Val Split and Tokenization

The ratio between train and val is 80:20

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        text = str(self.dataframe.iloc[index]['tweet'])
        label = int(self.dataframe.iloc[index]['class'])

        if pd.notna(text) and pd.notna(label):
          encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
          )

          return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
          }
        else:
            # Return an empty dictionary for skipped rows
            return {}

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define max sequence length based on your data
max_length = 128

# Create datasets and data loaders
train_dataset = CustomDataset(train_df, tokenizer, max_length)
val_dataset = CustomDataset(val_df, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Load pretrained model

In [ ]:
# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Send model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Model Training and Validation

In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Set up optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 3

for epoch in range(num_epochs):
    model.train()

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct_preds = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += criterion(outputs.logits, labels).item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += torch.sum(preds == labels).item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = correct_preds / len(val_loader.dataset)

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Loss: {loss:.4f}, '
          f'Validation Loss: {val_loss:.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}, '
          f'Precision: {precision:.4f}, '
          f'Recall: {recall:.4f}, '
          f'F1 Score: {f1:.4f}')


Epoch 1/3, Loss: 0.5065, Validation Loss: 0.2427, Validation Accuracy: 0.9118, Precision: 0.8951, Recall: 0.9118, F1 Score: 0.8901
Epoch 2/3, Loss: 0.2360, Validation Loss: 0.2577, Validation Accuracy: 0.9084, Precision: 0.8975, Recall: 0.9084, F1 Score: 0.9000
Epoch 3/3, Loss: 0.5384, Validation Loss: 0.2648, Validation Accuracy: 0.9114, Precision: 0.9062, Recall: 0.9114, F1 Score: 0.9084


## Save Model

In [9]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/MyDrive/CCCCC_bert

Mounted at /content/gdrive
/content/gdrive/MyDrive/CCCCC_bert


In [10]:
torch.save(model.state_dict(), 'model3.pth')

In [11]:
%cd ../../..

/content


## Load Model

In [ ]:
model.load_state_dict(torch.load('model3.pth', map_location=device))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Model Demo

In [13]:
cats = ""
model.eval()

# Example input text
input_text = "i wonder what happen to those guys"  #this is only an example of the input text to test the model

# Tokenize and encode the input text
inputs = tokenizer(input_text, return_tensors='pt')

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted class probabilities
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Get the predicted class label
predicted_class = torch.argmax(probs, dim=-1).item()

if predicted_class == 0:
  cats = "hate speech"
elif predicted_class ==1:
  cats = "offensive language"
elif predicted_class == 2:
  cats = "neither offensive nor non offensive"

print(f"Predicted Class: {predicted_class}, Category: {cats}, Probabilities: {probs.numpy()}")

Predicted Class: 2, Category: neither offensive nor non offensive, Probabilities: [[0.00839059 0.01282523 0.97878414]]
